<a href="https://colab.research.google.com/github/GautamV234/Summer-Intern-2021/blob/main/fnac_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat May 15 15:36:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |         

In [3]:
!pip install cellpose

     |████████████████████████████████| 13.3MB 204kB/s 
     |████████████████████████████████| 38.2MB 66kB/s 
  Created wheel for cellpose: filename=cellpose-0.6.5-cp37-none-any.whl size=152503 sha256=01884dd7ebfe2c17c6a8e3edcfb1b3fdb5a8e57060351b93041c9dc4728ec209
  Stored in directory: /root/.cache/pip/wheels/1c/df/8c/cc9a5bf8946adc663f419471d2ffe5879e37bc3bc6223adebf
Successfully built cellpose


In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
from cellpose import utils
from cellpose import models
import cv2
import glob
import pandas as pd
import h5py
import math
import time
from  PIL import Image
from scipy import ndimage
from skimage import exposure
from skimage import img_as_float
from skimage.morphology import watershed
from sklearn.metrics import pairwise
from skimage.feature import peak_local_max
from matplotlib import pyplot as plt
import os

from cellpose.utils import masks_to_outlines

In [5]:
use_GPU = models.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

2021-05-15 15:41:41,775 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? 1


In [6]:
cyto_model = models.Cellpose( model_type='cyto',gpu = True)

2021-05-15 15:45:02,104 [INFO] ** TORCH CUDA version installed and working. **
2021-05-15 15:45:02,105 [INFO] >>>> using GPU


In [10]:
def outline_img(img,mask,diameter):
    tom = img.copy()
    outl = masks_to_outlines(mask)
    a,b = np.nonzero(outl)
    tom[a, b] = np.array([255,255,255])
    h,w,c = tom.shape
    pad_a = np.full((h,15,3),[255,255,255],dtype = np.uint8)
    full = np.hstack((img,pad_a,tom))
    h_a,w_a,c_a = full.shape
    pad_b = np.full((50,w_a,3),[255,255,255],dtype = np.uint8)
    pad_b = cv2.putText(pad_b, 'Diameter_' + str(diameter) + '_Count_' + str(np.max(mask)),(10,40) , cv2.FONT_HERSHEY_SIMPLEX, 1 , (0,0,0), 3, cv2.LINE_AA )
    full_n = np.vstack((full,pad_b))
    return full_n

In [11]:
def run_one_model(images,channel,diameter):
    img_cop = images[:]
        
    cyto_masks, cyto_flow,_,_ = cyto_model.eval(list(img_cop), diameter= diameter, flow_threshold= 0.5,channels= [channel,0] ,augment= True , progress= False )
    
    frame_img = pd.DataFrame()
    frame_img['main_'+str(diameter)] = images
    frame_img['cyto_mask_'+str(diameter)] = cyto_masks
    frame_img['diameter'] = diameter
    frame_img['pred_out_'+str(diameter)] = frame_img.apply(lambda x: outline_img(x['main_'+str(diameter)], x['cyto_mask_'+str(diameter)],x['diameter']), axis=1)
    frame_img['cell_count_'+str(diameter)] = frame_img['cyto_mask_'+ str(diameter)].map(lambda x: np.max(x))
    
    del frame_img['cyto_mask_'+str(diameter)]
    del frame_img['main_'+str(diameter)]
    
    return frame_img

In [13]:
def run_all_params(images_data,image_names):
    
    small = run_one_model(images_data,3,20)
    auto = run_one_model(images_data,3,None)
    thrty = run_one_model(images_data,3,30)
    five_0 = run_one_model(images_data,3,50)
    hndrd = run_one_model(images_data,3,100)
    big = run_one_model(images_data,3,150)
    very_big = run_one_model(images_data,3,200)
    
    
    full = pd.concat([auto,small,thrty,five_0,hndrd,big,very_big] , axis = 1)
    full.index = image_names
    
    images_df = full.filter(like = 'pred_out_')
    cell_count_df = full.filter(like = 'cell_count_')
    
    return images_df,cell_count_df
    
    
    
    

In [14]:
def one_patient(input_directory,file_name):
    images = glob.glob(input_directory + '/*.jpg', recursive = True) 
    
    o_images = images
    o_images = pd.Series(o_images)
    image_name = o_images.map(lambda x:''.join(x.split('/')[-1].split('.')[:-1]))
    image_name = image_name[:]
    o_images = o_images.map(lambda x:cv2.imread(x))
    o_images = o_images.map(lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
    o_images = o_images.map(lambda x: cv2.resize(x,(512,512),interpolation = cv2.INTER_AREA))

  #lectin = pd.Series(images[1::2] )
  #lectin = lectin.map(lambda x:cv2.imread(x))
  #lectin = lectin.map(lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  
    images_df,cell_count_df =  run_all_params(o_images,image_name)
    return images_df,cell_count_df

In [15]:
all_files = [x for x in os.walk(r'/content/drive/MyDrive/fna_gautam/fna_40x/pap')][0][1]

In [16]:
all_files

['Ascitic fluid peritoneal carcinomatosis',
 'ADENO LUNG',
 'BAL FLUID ADENO',
 'ADENOCA GALL BLADDER',
 'BAL FLUID CA',
 'BAL BRUSHING LUNG  CA',
 'ADENOCA LUNG',
 'ASCITIC FLUID ADENO',
 'CA BREAST',
 'PTC',
 'UROTHELIAL CA',
 'LIVER CA',
 'MYO EPI CA ORBIT',
 'REACTIVE MESO',
 'METS SCC',
 'PLEURAL FLUID NHL',
 'PTC THYROID']

In [ ]:
for file in all_files:
    total = os.path.join(r'/content/drive/MyDrive/fna_gautam/fna_40x/pap',file)
    ano_path = os.path.join(r'/content/drive/MyDrive/fna_gautam/fna_pap_40_extract/images',file)
    ano_path_2 = os.path.join(r'/content/drive/MyDrive/fna_gautam/fna_pap_40_extract/excel',file)
    
    
    img_fr,cell_count_fr = one_patient(total,file)
    os.mkdir(ano_path_2)
    cell_count_fr.to_csv(ano_path_2 + '/report.csv')
    
    os.mkdir(ano_path)
    
    
    for leno in img_fr.index:
        img_path = os.path.join(ano_path,leno)
        os.mkdir(img_path)
        for each_diam in img_fr.columns:
            imgo = img_fr.loc[leno,each_diam]
            imgo = cv2.cvtColor(imgo,cv2.COLOR_RGB2BGR)
            
            img_path_ano = os.path.join(img_path,each_diam + '.jpg')
            cv2.imwrite(img_path_ano,imgo)
            
            
            
        
    

2021-05-15 15:46:55,002 [INFO] ~~~ FINDING MASKS ~~~
2021-05-15 15:46:55,008 [INFO] 0%|          | 0/18 [00:00<?, ?it/s]


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3829: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


2021-05-15 15:49:56,989 [INFO] 100%|##########| 18/18 [03:01<00:00, 10.11s/it]
2021-05-15 15:49:56,993 [INFO] >>>> TOTAL TIME 181.99 sec
2021-05-15 15:49:57,309 [INFO] ~~~ ESTIMATING CELL DIAMETER(S) ~~~
2021-05-15 15:50:58,796 [INFO] 100%|##########| 18/18 [01:01<00:00,  3.42s/it]
2021-05-15 15:50:58,806 [INFO] estimated cell diameter(s) in 61.50 sec
2021-05-15 15:50:58,808 [INFO] >>> diameter(s) = 
2021-05-15 15:50:58,814 [INFO] [25.73, 26.17, 21.59, 25.08, 22.37, 22.62, 24.55, 24.72, 20.54, 23.23, 21.85, 20.17, 22.65, 23.63, 21.62, 20.59, 26.87, 20.82, ]
2021-05-15 15:50:58,815 [INFO] ~~~ FINDING MASKS ~~~
2021-05-15 15:53:39,863 [INFO] 100%|##########| 18/18 [02:41<00:00,  8.95s/it]
2021-05-15 15:53:39,865 [INFO] >>>> TOTAL TIME 222.56 sec
2021-05-15 15:53:40,185 [INFO] ~~~ FINDING MASKS ~~~
2021-05-15 15:55:16,480 [INFO] 100%|##########| 18/18 [01:36<00:00,  5.35s/it]
2021-05-15 15:55:16,482 [INFO] >>>> TOTAL TIME 96.30 sec
2021-05-15 15:55:16,787 [INFO] ~~~ FINDING MASKS ~~~
2021